# Предобработка данных

In [ ]:
## Предобработка данных
# что включить из предобработки:
# 1. выделение русскоязычной лексики из песен
# 2. лемматизация

In [ ]:
# тетрадка загружалась на Kaggle, тк ввиду большого объёма датасеты хранятся там

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import collections
import pandas_profiling
import re
import string

In [ ]:
df = pd.read_csv("/kaggle/input/famous-ru-songs-1985-2023/df_1985_2023_top_titles.csv", encoding='utf8')

In [ ]:
puncts = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—«»…'

## Выделяем русскоязычные тексты песен в отдельный стобец lyrics_ru

In [ ]:
#добавляем новый столбец по индексу
df.insert(13, 'lyrics_ru', 'NaN')

In [ ]:
n = 0
for song in df['lyrics']:
    song = str(song)
    ru_lyrics_list = []
    not_ru_lyrics_list = []

    for word in song.split():
        word = word.strip(puncts)
        try:
            ru_lyrics = re.search(r'(([А-Я|Ё]{0,}\s*([а-я|ё]+\s*)*)+)', word).groups()[0]
        except:
            pass

        try:
            if len(ru_lyrics) > 0:
                ru_lyrics_list.append(ru_lyrics)
            else:
                pass
        except:
            pass

    df['lyrics_ru'].iloc[n] = " ".join(ru_lyrics_list)

    n += 1

## Названия песен

Названия песен изначально были представлены в одной колонке на русском и английском языках, задачей было разбить компоненты по разным ячейкам


In [ ]:
n = 0
for elem in df_1985_2023['title']:
    lang = str()
    try:
        new_elem = re.sub(r'[^\w\s]', ' ', df['title'][n])
    except:
        pass

    try:
        ru_name = re.search(r'(([А-Я|Ё]{0,}\d*\s*([а-я|ё]+\d*\s*)*)+)', new_elem).groups()[0]
        df_1985_2023['ru_title'].iloc[n] = str(ru_name)

        lang += 'ru '
    except:
        pass

    try:
        en_name = re.search(r'(([A-Z]{0,}\d*\s*([a-z]+\d*\s*)+)+)', new_elem).groups()[0]
        df_1985_2023['en_title'].iloc[n] = str(en_name)
        lang += 'en '
    except:
        pass

    df_1985_2023['title_lang'].iloc[n] = lang

    n += 1

## Лемматизация

In [ ]:
!pip install pymystem3

In [ ]:
from pymystem3 import Mystem

### Лемматизация текста

In [ ]:
#лемматизируем текст и добавим столбец с длиной песени
n = 0

for song in df['lyrics_ru']:
    song = str(song)
    m = Mystem()
    lemmas = m.lemmatize(song)

    df['lemmas_ru_text'].iloc[n] = " ".join(lemmas)

    n += 1

### Лемматизация названий

In [ ]:
n = 0
k = 0
for song in df['ru_title']:
    try:
        song = str(song)
        m = Mystem()
        lemmas = m.lemmatize(song)

        df['lemmas_ru_title'].iloc[n] = " ".join(lemmas)

        n += 1
    except:
        pass

### Стобец tags_title не анализировался в работе, но представлен в датасете

В него вошли грамматические тэги (pymorphy), свойственные словоформам в названиях

In [ ]:
df.insert(7, 'tags_title', 'NaN')

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
morph_ru = pymorphy2.MorphAnalyzer(lang='ru')

In [ ]:
n = 0

for word in df['ru_title'][n].split():
    try:
        tags = []

        p = morph.parse(word)[0]
        tag = p.tag
        tags.append(str(tag))

        df['tags_title'][n] = ' '.join(tags)

    except:
        pass

    n+=1

## Лемматизация с удалением стоп-слов

In [ ]:
with open('/content/drive/My Drive/spil_lemmas/ru_stopwords.txt') as f:
    stopwords = f.read().split()

In [ ]:
df.insert(19, 'lemmas_ru_text_nostopwords', 'NaN')

In [ ]:
n = 0
k = 0

for song in df['lemmas_ru_text'][0:]:
    no_stopwords_text = []

    try:
        if type(df['lemmas_ru_title'][n]) == str:
            song_with_title = df['lemmas_ru_title'][n] + ' ' + df['lemmas_ru_text'][n]
        else:
            song_with_title = str(song)
    except:
        pass


    for word in song_with_title.split():
        if word not in stopwords:
            no_stopwords_text.append(word)
        else:
            pass


    df['lemmas_ru_text_nostopwords'].iloc[n] = " ".join(no_stopwords_text)


    n += 1

## Удаление столбцов, не релевантных для работы

In [ ]:
df.drop('language_cld3', inplace=True, axis=1)
df.drop('language_ft', inplace=True, axis=1)
df.drop('language_id', inplace=True, axis=1)

In [ ]:
# df.to_csv('df_lemmas_upd.csv', encoding='utf8')